Consider the following classes:

In [1]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, vol):
        self.symbol = symbol
        self.date = date
        self.open_ = open_
        self.high = high
        self.low = low
        self.close = close
        self.vol = vol
        
    def as_dict(self):
        return dict(symbol = self.symbol,
                    date = self.date,
                    open = self.open_,
                    high = self.high,
                    low = self.low,
                    close = self.close,
                    vol = self.vol)

In [2]:
class Trade:
    def __init__(self, symbol, timestamp, order, price, vol, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.vol = vol
        self.commission = commission
        
    def as_dict(self):
        return dict(symbol = self.symbol,
                    timestamp = self.timestamp,
                    order = self.order,
                    price = self.price,
                    vol = self.vol,
                    commission = self.commission)

### Exercise 1:

Write a custom `JsonEncoder` class to serialize dictionaries that contain instances of the above classes.

In [3]:
import json
from pprint import pprint
from datetime import date, datetime
from decimal import Decimal

In [4]:
class CustomJsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            result = obj.as_dict()
            result['object'] = obj.__class__.__name__
            return result
        if isinstance(obj, Trade):
            return obj.as_dict()
        if isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        if isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        if isinstance(obj, Decimal):
            return str(obj)
        return super().default(obj)

In [5]:
activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.25'), 365_607),
        Stock('AAPL', date(2018, 11, 22), Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_669_184),
        Stock('MSFT', date(2018, 11, 22), Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.23'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [6]:
encoded = json.dumps(activity, cls=CustomJsonEncoder)
pprint(encoded)

('{"quotes": [{"symbol": "TSLA", "date": "2018-11-22", "open": "338.19", '
 '"high": "338.64", "low": "337.60", "close": "338.25", "vol": 365607, '
 '"object": "Stock"}, {"symbol": "AAPL", "date": "2018-11-22", "open": '
 '"176.66", "high": "177.25", "low": "176.64", "close": "176.78", "vol": '
 '3669184, "object": "Stock"}, {"symbol": "MSFT", "date": "2018-11-22", '
 '"open": "103.25", "high": "103.48", "low": "103.07", "close": "103.11", '
 '"vol": 4493689, "object": "Stock"}], "trades": [{"symbol": "TSLA", '
 '"timestamp": "2018-11-22", "order": "buy", "price": "338.23", "vol": 100, '
 '"commission": "9.99", "object": "Trade"}, {"symbol": "AAPL", "timestamp": '
 '"2018-11-22", "order": "sell", "price": "177.01", "vol": 20, "commission": '
 '"9.99", "object": "Trade"}]}')


---

### Exercise 2:

Write a custom decoder that can deserialize a JSON structure containing `Stock` and `Trade` objects.

In [7]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, encoded):
        data = json.loads(encoded)
        return self.parse(data)
    
    def parse(self, obj):
        # Loaded object can be either list or dictionary
        if isinstance(obj, dict):
            obj = self.decode_financials(obj)
            if isinstance(obj, dict):
                for key, value in obj.items():
                    obj[key] = self.parse(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.parse(item)
        return obj
                
    def decode_financials(self, obj):
        obj_type = obj.get('object', None)
        if obj_type == 'Stock':
            return self.decode_stock(obj)
        elif obj_type == 'Trade':
            return self.decode_trade(obj)
        return obj
    
    def decode_stock(self, obj):
        s = Stock(obj['symbol'],
                  datetime.strptime(obj['date'], '%Y-%m-%d').date(),
                  Decimal(obj['open']),
                  Decimal(obj['high']),
                  Decimal(obj['low']),
                  Decimal(obj['close']),
                  int(obj['vol']))
        return s
        
    def decode_trade(self, obj):
        t = Trade(obj['symbol'],
                  datetime.strptime(obj['timestamp'], '%Y-%m-%d'),
                  obj['order'],
                  Decimal(obj['price']),
                  int(obj['vol']),
                  Decimal(obj['commission']))
        return t

In [8]:
decoded = json.loads(encoded, cls=CustomDecoder)
print(decoded)

{'quotes': [<__main__.Stock object at 0x000001F32CB868E0>, <__main__.Stock object at 0x000001F32CB86880>, <__main__.Stock object at 0x000001F32CB86070>], 'trades': [<__main__.Trade object at 0x000001F32CB869A0>, <__main__.Trade object at 0x000001F32CB86910>]}
